# 네이버 영화 평점 Crawling by 민또

### 기본적인 구조 파악

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req
import math

In [2]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
num = 158885   ### 얘를 바꿔주면 댐
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
page = 1   ### 한 페이지당 평점 10개

response = req.urlopen(url1+str(num)+url2+str(page))
print(response)

In [5]:
soup = BeautifulSoup(response, 'html.parser')   # 소스 파싱
score_result = soup.find('div', class_ = 'score_result')   # 리뷰와 평점이 있는 부분만 자르기
lis = score_result.find_all('li')    # li 별로 잘라서 벡터형태로 변경

In [ ]:
print(soup)   ### 페이지 전체 소스

In [ ]:
print(lis[0])   # 여기서 코멘트만 추출해야 함

In [6]:
### 페이지 한 창의 평점 크롤링

for li in lis:  # 각 li 마다 평점과 코멘트 추출
    score = li.select_one("div.star_score > em").get_text()   ### 평점 추출
    reple = li.select_one("div.score_reple > p").get_text()   ### 리뷰 추출
    print('평점 :',  score)
    print('리뷰 :', reple, '\n')

평점 : 10
리뷰 : BEST소년의 첫사랑,  마지막즈음 아버지의 대사가 인상 깊었고    나중에 올리버 시점으로 나왔으면,,   

평점 : 10
리뷰 : BEST전화 목소리를 잘 분간하지 못하던 소년은 반년만에 들려온 그의 목소리는 단박에 알아듣는다. 오랜만에 인물의 감정 속에 푹 빠져 본 영화였다.   

평점 : 10
리뷰 : BEST이 영화의 아름다움을 느끼지 영원히 못하고 평점테러중인 호모포비아들이 애잔함   

평점 : 9
리뷰 : BEST'우린 서른살이 되기 전에 이미 무너져.새로운 사람을 만나도 더이상 보여줄 내가 없어지게 되지.'실제로 저런 조언을 해주는 아버지가 몇이나 될것이며 저런 시선으로 바라봐주는 세상이 존재는 할까.영화가 너무 꿈같아서 눈물겹도록 아름답고 슬프다고해야되나.   

평점 : 10
리뷰 : BEST첫사랑을 느낄 수 있는 사랑과 아픔을 너무 잘 표현해냈고모든 장면, 모든 대사 하나하나가 너무 아름다웠다6주간의 추억으로 평생을 살아갈 엘리오나는 그 누군가를 이토록 간절히 원한 적 있었던가....   

평점 : 10
리뷰 : 남자 대 남자가 아닌 사람 대 사람의 첫사랑의 열병. 마지막 엔딩 엘리오의 표정에서 올리버와의 첫만남부터 헤어짐까지의 모든것이 담겨있다.   

평점 : 9
리뷰 : 추운날 빈좌석 많은 상영관에 홀로앉아 이탈리아로 바캉스 다녀온다 생각하고 보면 좋을 영화. 이 영화가 돋보이는 가장 큰 이유는 가장 완벽한 어른, 가장 불안정한 아이, 그리고 그럴만한 현실이  어우러진  첫사랑 영화라는 점이다.   

평점 : 10
리뷰 : 아 첫사랑이 첫사랑으로만 끝나야했지만 그것을 간직한 채 평생을 살아갈 엘리오의 마지막 씬 때문에 조조로 영화 보면서 울었네요,,   

평점 : 10
리뷰 : 관람객Because I wanted you to know   

평점 : 10
리뷰 : 관람객첫사랑의 기억을 통째로 소환한다  안흔들리고 못배길 영화   



### for 문을 사용하여 페이지와 영화종류를 바꿔가며 출력해보자!

In [4]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
movies = [85578, 136898]   ### 일단 두개의 영화만 돌릴거임~
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='



for i in movies:
    num = i
    response = req.urlopen('https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(num))   ### 영화제목
    soup = BeautifulSoup(response, 'html.parser')   ### 소스 파싱
    title =  soup.select_one("title").string   ### 영화 제목 추출(근데 봐서 이건 빼도 될듯...)
    print('###', title, '###', '\n')   ### 젤 첨에 영화 제목 출력
    
    for j in range(1, 4):   ### 1페이지부터 3페이지까지 돌림
        page = j
        response = req.urlopen(url1+str(num)+url2+str(page))
        soup = BeautifulSoup(response, 'html.parser')
        score_result = soup.find('div', class_ = 'score_result')
        lis = score_result.find_all('li')   # 리뷰가 담겨있는 각 마디를 잘라 벡터화
        
        for li in lis:
            score = li.select_one("div.star_score > em").get_text()
            reple = li.select_one("div.score_reple > p").get_text()
            print(score, ' , ', reple, '\n')   ### 평점, 리뷰 출력

### 7년의 밤 : 네이버 영화 ### 

1  ,  BEST염력도 피하고 골든슬럼버도 피했는데 이걸 못피함   

8  ,  BEST배우들의 열연과 몰입도는 좋았지만 2시간 내내 똑같이 하드하기만 해서 힘들었음.. 완급을 주는 시나리오였음 좋았을듯   

9  ,  BEST관람객장동건, 류승룡의 연기도 좋았지만 난 송새벽에게 한표 주고 싶음. 제 3자로써 이 이야기를 가장 많이 알고 있는 인물이었기 때문에 더 연기하기가 힘들었을텐데 정말 자연스럽게 잘하더라   

8  ,  BEST관람객장동건...연기 소름 오래전 읽었던 정유정 작가의 책을 다시금 펼쳐 읽어야겠다   

8  ,  BEST관람객책으로보고기대했는데 생각보다 긴장감도 있고재밌었습니다.   

4  ,  7년의밤이 아나라 2시간의밤임 2시간동안 밤처럼앞이 안보임   

2  ,  무슨 얘기를 하자는 건지 태어나서 처음 평가한다 답답해서   

9  ,  장동건 연기를 이렇게 잘하나 싶었다   

10  ,  장동건 원래 연기 잘 했죠.. 작품들이 안 좋아서 그렇지   

1  ,  7년 같았던 2시간..   

10  ,  관람객세령마을 싱크 최고였어요ㅠ분위기와 배우들 연기로 압도   

10  ,  연기 몰입감 최고네 ㅎㄷㄷ;;   

9  ,  소설은 읽지 않았지만  연기 출연진 굿   

3  ,  오영제 부인은 결국뭐죠? 류승룡 부인은 댐에 쓸려 그냥죽었나요? 뭔 이런내용의 영화를만들수있는지.. 뭘말하자는건지 모르겠네요 아들을 지키기위해 마을사람들을 죽일수밖에 없었던 가장의 부성애? 그나마 장동건 류승룡 연기때문에 3점 드립니다   

1  ,  왜 류승룡은 영화가 항상 이렇게 망하는가...   

8  ,  관람객약간 아쉽지만 나름좋앗다ㅡ하이라이트인ㅡ댐에서ㅡ두남자의 난투극은 잘찍엇네ㅡcg없이 댐으로ㅡ사다리타고ㅡ올라가는장면을 롱테이크로ㅡ찍은건 압권이엇딘   

1  ,  7년동안 다시는 이런 영화 안 봤으면 좋겠네요~~상영시간내내 시계 확인 ㅎㅎ   

10  ,  관람객스릴러 드라마 장

### 이제 크롤링한 내용을 텍스트파일로 내보내보자!

In [13]:
url1 = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='
movies = [85578, 85579, 99354, 123519, 125494, 134963, 136898, 137008, 137326, 144379, 151728, 153620, 154449, 154598, 158626, 158885, 161242, 165026, 168298, 172454]
### 7년의 밤, 신과함께, 지슬, 아가씨, 퍼시빅림, 라라랜드, 레디플레이어원, 리얼, 블랙팬서, 러빙빈센트, 
### 코코, 치인트, 리틀포레스트, 그레이50, 셰잎옾워터, 콜미바이유어네임, 범죄도시, 사라진밤, 지금만나러갑니다, 곤지암
url2 = '&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='


for i in movies:
    f = open('Movie_'+str(i)+'.txt', 'w')   ### 영화 한개 마다 txt 파일로 저장
    num = i
    response = req.urlopen('https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(num))
    soup = BeautifulSoup(response, 'html.parser')
    title = soup.select_one("title").string
    f.write('### '+title+' ###'+'\n\n')   ### 영화 제목 출력
    
    response = req.urlopen(url1+str(num)+url2+'1')   ### 댓글창 개수를 뽑아내보자
    soup = BeautifulSoup(response, 'html.parser')    ### 소스 파싱
    score_total = soup.find('div', class_ = 'score_total')    ### 평점 개수 나타난 곳 추출
    limit = score_total.find_all('em')[1]   ### 평점 개수 추출
    page_limit = int(limit.string.replace(',', ''))/10   ### 숫자형식으로 바꿔서 10으로 나눠줌
    page_limit = math.ceil(page_limit)   ### 소숫점 올림

    for j in range(1, page_limit):
        page = j
        response = req.urlopen(url1+str(num)+url2+str(page))
        soup = BeautifulSoup(response, 'html.parser')
        score_result = soup.find('div', class_ = 'score_result')
        lis = score_result.find_all('li')   # 리뷰가 담겨있는 각 마디를 잘라 벡터화
        
        for li in lis:
            score = li.select_one("div.star_score > em").get_text()
            reple = li.select_one("div.score_reple > p").get_text()
            f.write(score+' , '+reple+'\n')   ### 평점, 리뷰 출력
    f.close   ### 텍스트 입력 종료

### 대략 2시간동안 22만개 출력  (1분에 2000개 꼴..)